Importing the fastai libary for image recognition and others used in the model

In [ ]:
from fastai.vision.all import *
from torchvision.models import *
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

Here I create paths to the different data in my dataset

In [ ]:
data_path = Path('/kaggle/input/chexpert')
path_train = Path('/kaggle/input/chexpert/CheXpert-v1.0-small/train')
path_valid = Path('/kaggle/input/chexpert/CheXpert-v1.0-small/valid')

Gets the csv files for training and validation

In [ ]:
train_df = pd.read_csv(data_path/'CheXpert-v1.0-small/train.csv')
valid_df = pd.read_csv(data_path/'CheXpert-v1.0-small/valid.csv')

Looking at what the csv files are containing 

In [ ]:
train_df.head()

In [ ]:
valid_df.head()

Exploring the data for NaN values

In [ ]:
train_df.isna().sum()

In [ ]:
valid_df.isna().sum()

Making a list of the targets we are looking to find

In [ ]:
    chexnet_targets = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

Handles the NaN values by filling them in with the value 0, and making any uncertain answers to 1 (this is not the best solution but the one we got time for, when working with this project later this will be a focus point in making a better model)

In [ ]:
train_df = train_df.fillna(0)
train_df[chexnet_targets] = train_df[chexnet_targets].abs()

In [ ]:
valid_df = valid_df.fillna(0)

Look over the data after filling in NaN values and converting the -1 values to 1

In [ ]:
train_df.head()

In [ ]:
valid_df.head()

Making the functions for getting the pictures path (get_x) that will be used by our vision learner

Making the function for finding wich condition the patient has (get_y)

In [ ]:
def get_x(row): 
    return data_path / row['Path']

def get_y(row):
    condition_columns = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']
    labels = [i for i in condition_columns if row[i]== 1]
    return labels

Combining the csv files into one singular csv file, but making sure to mark what was in the validation set so that we can use this later

In [ ]:
combined_df = pd.concat([train_df, valid_df], ignore_index=True)

valid_idx_start = len(train_df)

valid_idx = list(range(valid_idx_start, len(combined_df)))

Making the datablock that will be used later

In [ ]:
datablock = DataBlock(
    blocks = (ImageBlock, MultiCategoryBlock),
    splitter = IndexSplitter(valid_idx),
    get_x = get_x,
    get_y = get_y,
    item_tfms = Resize(256),
    batch_tfms = aug_transforms(size=224, min_scale=0.75)
)

In [ ]:
def get_dataloaders(batch_size):
    # Definer DataBlock og dataloaders her basert på ditt datasett
    # For eksempel:
    datablock = DataBlock(
        blocks=(ImageBlock, MultiCategoryBlock),
        splitter=IndexSplitter(valid_idx),  # Juster splitteren etter behov
        get_x=get_x,
        get_y=get_y,
        item_tfms=Resize(256),
        batch_tfms=aug_transforms(size=224, min_scale=0.75)
    )

    dls = datablock.dataloaders(combined_df, bs=batch_size)
    return dls

def create_model(num_layers):
    # Opprett modellen basert på ønsket arkitektur (f.eks. resnet) og num_layers
    if num_layers == 18:
        model = resnet18(pretrained=True)
    elif num_layers == 34:
        model = resnet34(pretrained=True)
    elif num_layers == 50:
        model = resnet50(pretrained=True)
    else:
        raise ValueError("Ugyldig num_layers valgt")

    # Tilpasninger av modellen kan legges til her hvis nødvendig
    # For eksempel: Tilpasning av siste lag for antall klasser i ditt datasett
    num_classes = len(dls.vocab)
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    return model


Making the dataloader with the new csv file and showing three images for one of the batches

In [ ]:
dls = datablock.dataloaders(combined_df, bs=32)
dls.show_batch(nrows=3, ncols=1)

Training the model on out dataset, here we choose to use the resnet18 pre-trained model. For metrics we used the accuracy_multi that will give a accuracy for each condition and we set a threshold of 0,5 so that the model will only say the condition is present if it is 50% or more sure that it is present

Under we find a learning rate and we get three different options and choose the one that looks the best

In [ ]:
learn = vision_learner(dls, resnet18, metrics=partial(accuracy_multi, thresh=0.5))
lr = learn.lr_find(suggest_funcs=(valley, steep, slide))

In this code, we are using the fastai library along with the Hyperopt package to perform hyperparameter tuning for a deep learning model. We define a training and validation function using fastai, specify a hyperparameter search space including learning rate, number of layers, and batch size, and then utilize the Hyperopt fmin function to search for the best hyperparameters within this space. Finally, we train the model using the best hyperparameters found and save the trained model for future use.

In [ ]:
from fastai.vision.all import *
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from functools import partial




def train_model(params):
    
    learn.fine_tune(1, base_lr=params['learning_rate'])
    loss = learn.recorder.values[-1][0]
    return {'loss': loss, 'status': STATUS_OK}

   
    space = {
    'learning_rate': hp.loguniform('learning_rate', np.log(1e-5), np.log(1e-2)),
    'num_layers': hp.choice('num_layers', [18, 34, 50]),
    'batch_size': hp.choice('batch_size', [16, 32, 64]),
    }

    
    trials = Trials()
    best_params = fmin(train_model, space, algo=tpe.suggest, max_evals=10, trials=trials)

print("Beste hyperparametere:")
print(best_params)


best_dls = get_dataloaders(best_params['batch_size'])
best_learn = create_model(best_params['num_layers'])
best_learn.fine_tune(1, base_lr=best_params['learning_rate'])
import pickle
pickle.dump(best_learn, open("cheXpertModel", "wb"))







Here we present the cases with the higghest loss in our validation set. We get to see what conditions the patient had but also what the model predictet in additon to the different probabilities

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(5, figsize=(15,10))

In the next cells we set up a multilabel condusion matrix to se what conditions the model preforms well at and where the model preforms bad.

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

# Get predictions
preds, targs = learn.get_preds()

# Convert predictions to binary using a threshold (e.g., 0.5)
binary_preds = preds > 0.5

# Calculate confusion matrix for each label
cm = multilabel_confusion_matrix(targs, binary_preds)

In [ ]:
import seaborn as sns

def plot_confusion_matrix(cm, label, ax=None):
    """
    Plots a confusion matrix using seaborn's heatmap.
    """
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_title(f'Confusion Matrix for {label}')
    ax.set_xlabel('Predicted Labels')
    ax.set_ylabel('True Labels')
    ax.xaxis.set_ticklabels(['Negative', 'Positive'])
    ax.yaxis.set_ticklabels(['Negative', 'Positive'])

# Plot the confusion matrix for each label
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(20, 20))
axes = axes.flatten()
for i, label in enumerate(chexnet_targets):
    plot_confusion_matrix(cm[i], label, ax=axes[i])
plt.tight_layout()
plt.show()


In [ ]:
def grad_cam(learn, x, class_id=None):
    # Get the model's last convolutional layer
    target_layer = learn.model[0][-1]
    
    # Hook the feature extractor
    features = {}
    def get_features_hook(module, input, output):
        features["conv_features"] = output.detach()
    
    handle_features = target_layer.register_forward_hook(get_features_hook)
    
    # Hook for the gradients
    gradients = {}
    def get_gradients_hook(module, grad_input, grad_output):
        gradients["conv_gradients"] = grad_output[0].detach()
    
    handle_gradients = target_layer.register_backward_hook(get_gradients_hook)
    
    # Forward pass
    output = learn.model.eval()(x)
    if class_id is None:
        class_id = output.argmax(dim=1).item()
    
    # Zero gradients
    learn.model.zero_grad()
    
    # Backward pass for the selected class
    target = output[:, class_id]
    target.backward()
    
    # Get the features and gradients
    conv_features = features["conv_features"]
    conv_gradients = gradients["conv_gradients"]
    
    # Pool the gradients across the channels
    pooled_gradients = torch.mean(conv_gradients, dim=[0, 2, 3])
    
    # Weight the channels by corresponding gradients
    for i in range(conv_features.shape[1]):
        conv_features[:, i, :, :] *= pooled_gradients[i]
    
    # Average the channels of the features
    heatmap = torch.mean(conv_features, dim=1).squeeze()
    heatmap = np.maximum(heatmap.cpu().numpy(), 0)
    heatmap /= np.max(heatmap)
    
    # Cleanup hooks
    handle_features.remove()
    handle_gradients.remove()
    
    return heatmap


def plot_heatmap(heatmap, ax, alpha=0.6):
    img = TensorImage(dls.train.decode((x,))[0][0])  # Decode to get the image
    img.show(ctx=ax)  # Show the image
    ax.imshow(heatmap, alpha=alpha, extent=(0, 224, 224, 0),
              interpolation='bilinear', cmap='magma')

# Usage
x, _ = dls.one_batch()
x = x[0].unsqueeze(0).cuda()  # Use the first image of the batch and ensure it's on GPU
class_id = None  # Auto-select the class based on model prediction or specify it

heatmap = grad_cam(learn, x, class_id=class_id)

_, ax = plt.subplots()
plot_heatmap(heatmap, ax)



In [ ]:
def grad_cam_batch(learn, xb, class_ids=None):
    """
    Generate Grad-CAM heatmaps for a batch of images.
    
    Args:
    learn (Learner): The fastai learner with a convolutional model.
    xb (Tensor): A batch of images, expected shape [N, C, H, W].
    class_ids (list of int, optional): List of class IDs for which to generate the Grad-CAM.
        If None, the class with the highest output score is used for each image.
    
    Returns:
    List[Tensor]: A list of heatmaps for each image in the batch.
    """
    # Ensure model is in evaluation mode and move input to same device as model
    m = learn.model.eval()
    xb = xb.to(learn.dls.device)

    # Hook the feature extractor
    features = {}
    def hook_features(module, input, output):
        features['conv_features'] = output.detach()

    # Hook the gradients
    gradients = {}
    def hook_gradients(module, grad_in, grad_out):
        gradients['conv_gradients'] = grad_out[0].detach()

    # Register hooks on the last convolutional layer
    hook_f = m[0][-1].register_forward_hook(hook_features)
    hook_b = m[0][-1].register_backward_hook(hook_gradients)

    # Forward pass to get model predictions
    preds = m(xb)
    if class_ids is None:
        # If no class IDs provided, use the class with the highest output score
        class_ids = preds.argmax(dim=1).tolist()

    heatmaps = []
    for i, class_id in enumerate(class_ids):
        one_hot = torch.zeros_like(preds[i])
        one_hot[class_id] = 1
        preds[i].backward(one_hot, retain_graph=True if i < len(class_ids)-1 else False)

        # Get features and gradients for the current image
        conv_features = features['conv_features'][i].unsqueeze(0)
        conv_gradients = gradients['conv_gradients'][i].unsqueeze(0)

        # Pool the gradients across the channels and weight the channels
        pooled_gradients = torch.mean(conv_gradients, dim=[0, 2, 3])
        weighted_features = torch.zeros_like(conv_features)
        for j in range(conv_features.shape[1]):
            weighted_features[:, j, :, :] = conv_features[:, j, :, :] * pooled_gradients[j]

        # Generate heatmap by averaging across channels and applying ReLU
        heatmap = torch.mean(weighted_features, dim=1).squeeze()
        heatmap = F.relu(heatmap)
        heatmap /= torch.max(heatmap)
        heatmaps.append(heatmap.cpu())

    # Remove hooks
    hook_f.remove()
    hook_b.remove()

    return heatmaps, class_ids


In [ ]:
def plot_batch_with_heatmaps(dls, xb, heatmaps, class_ids):
    """
    Plots a batch of images with their corresponding Grad-CAM heatmaps.
    
    Args:
    dls (DataLoaders): The fastai dataloaders, used for decoding batch.
    xb (Tensor): The batch of images.
    heatmaps (list of Tensor): A list of heatmaps corresponding to xb.
    """
    
    nrows = len(xb)
    ncols = 2  # For each row: [Original Image, Heatmap]
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*5, nrows*5))
    
    for i in range(nrows):
        # Decode and show original image
        img = dls.train.decode((xb[i:i+1],))[0][0]
        img.show(ctx=axs[i, 0] if nrows > 1 else axs[0])
        axs[i, 0].set_title('Original Image')
        
        # Show heatmap
        axs[i, 1].imshow(img.permute(1, 2, 0).cpu().numpy())
        axs[i, 1].imshow(heatmaps[i], alpha=0.5, extent=(0, img.shape[2], img.shape[1], 0),
                         cmap='jet')
        # Use class_id for the title if available, can be replaced with class names if you have a mapping
        class_name = dls.vocab[class_ids[i]]
        axs[i, 1].set_title(f'Grad-CAM: Class {class_name}')
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Get a batch of images
xb, _ = dls.one_batch()

# Generate class IDs if you have specific targets, else None to use the highest scoring class
class_ids = None  # Example: [0, 1, 2] for the first three classes, if specific classes are targeted

# Generate heatmaps
heatmaps, class_ids = grad_cam_batch(learn, xb[:3], class_ids=class_ids)  # Using the first 3 images of the batch

# Visualize
plot_batch_with_heatmaps(dls, xb[:3], heatmaps, class_ids)


In [ ]:
import pickle
pickle.dump(learn, open("cheXpertModel", "wb"))

In [ ]:
model_loaded = pickle.load(open("cheXpertModel", "rb"))

In [ ]:
interp = ClassificationInterpretation.from_learner(model_loaded)
interp.plot_top_losses(5, figsize=(15,10))